In [158]:
from Preprocess import *
from Model import *
from DataAnalysis import *

In [159]:
dic_csv = read_data()
df_var = make_dataframe(dic_csv)

In [ ]:
concat_train_test(dic_csv, df_var)
loans = Merge(df_var, haveCategorical=True)

In [ ]:
original_loan_per_year = loan_per_year(df_var['loan'], isMerged = False)

In [ ]:
histDf(original_loan_per_year)

In [ ]:
getCorr(original_loan_per_year[1993])

In [ ]:
model_year(original_loan_per_year[1993], original_loan_per_year[1994])


In [182]:
client = df_var['client']
age = getAge(client)
#ax = age.plot.hist(bins=12, alpha=0.5)

ValueError: 1 columns passed, passed data had 2 columns

In [178]:

def getAge(df):
    list = []
    for row in df.itertuples(index = True):
        d = getDate(row.birth_number)
        birth_number = row.birth_number 
        list.append([row.client_id, d['year']])

    return pd.DataFrame(list, columns=['client_id' 'year'])